# Covid19 Swedish Trend Map

In [657]:
from urllib.request import urlopen
import json
#Example data
with open('features.geojson') as response:
    counties = json.load(response)

In [658]:
counties['features'][0]

{'type': 'Feature',
 'properties': {'ISO': 'SWE',
  'NAME_0': 'Sweden',
  'ID_1': 4,
  'NAME_1': 'Gävleborg',
  'HASC_1': 'SE.GV',
  'TYPE_1': 'Laen|län',
  'ENGTYPE_1': 'County',
  'VARNAME_1': 'Gävleborgs län'},
 'geometry': {'coordinates': [[[17.231615, 61.044258],
    [17.252025, 61.037439],
    [17.237502, 61.020941],
    [17.206887, 61.03282],
    [17.231615, 61.044258]]],
  'type': 'Polygon'},
 'id': '039e3e653bd01f2744b1bb6f50fd2d7b'}

In [659]:
len(counties['features'])

105

In [660]:
ids = []
regions = []
for i in range(len(counties['features'])):
    ids.append(counties['features'][i]['id'])
    regions.append(counties['features'][i]['properties']['NAME_1'])

In [661]:
df = pd.DataFrame([ids,regions]).T

In [662]:
df.columns = ['id','NAME_1']
df['NAME_1'] = df['NAME_1'].replace({'Orebro':'Örebro','Jämtland':'Jämtland Härjedalen','Södermanland':'Sörmland'})

In [663]:
df_reg = pd.read_pickle('regiondata2')

print(df_reg.reset_index().Metric.unique())

df_cases = df_reg['Totalt_antal_avlidna'].reset_index()
df_mobility = df_reg['Mobility'].reset_index()
df_tests = df_reg['Totalt_antal_fall'].reset_index()
df_icu = df_reg['Totalt_antal_intensivvårdade'].reset_index()

['Totalt_antal_fall' 'Fall_per_100000_inv' 'Totalt_antal_intensivvårdade'
 'Totalt_antal_avlidna' 'New_cases' 'Mobility']


In [664]:
df_icu

df_icu.columns = ['NAME_1','Date','ICU_Patients']
df_icu = df_icu.drop('Date',axis=1)

In [665]:
df_tests

df_tests.columns = ['NAME_1','Date','Tested_Confirmed']
df_tests = df_tests.drop('Date',axis=1)

In [666]:
df_cases

df_cases.columns = ['NAME_1','Date','Deaths']
df_cases = df_cases.drop('Date',axis=1)

In [667]:
df_mobility = df_mobility[df_mobility['Date']==df_mobility.Date.unique()[-1]]
df_mobility.columns = ['NAME_1','Date','Mobility']
df_mobility["Mobility"] = df_mobility["Mobility"].replace(',','.',regex=True)

In [668]:
df_mobility['Mobility'] = pd.to_numeric(df_mobility["Mobility"], downcast="float")
df_mobility = df_mobility.drop('Date',axis=1)
df_mobility['NAME_1'] = df_mobility['NAME_1'].replace({'Gavleborg':'Gävleborg','Jamtland':'Jämtland Härjedalen','Jönkoping':'Jönköping'})

In [669]:
df = df.set_index('NAME_1').join(df_cases.set_index('NAME_1')).reset_index()

In [670]:
df = df.set_index('NAME_1').join(df_mobility.set_index('NAME_1')).reset_index()

In [671]:
df = df.set_index('NAME_1').join(df_tests.set_index('NAME_1')).reset_index()

In [672]:
df = df.set_index('NAME_1').join(df_icu.set_index('NAME_1')).reset_index()

In [700]:
import plotly.express as px

print(df["NAME_1"][2])
print(counties["features"][0])

Blekinge
{'type': 'Feature', 'properties': {'ISO': 'SWE', 'NAME_0': 'Sweden', 'ID_1': 4, 'NAME_1': 'Gävleborg', 'HASC_1': 'SE.GV', 'TYPE_1': 'Laen|län', 'ENGTYPE_1': 'County', 'VARNAME_1': 'Gävleborgs län'}, 'geometry': {'coordinates': [[[17.231615, 61.044258], [17.252025, 61.037439], [17.237502, 61.020941], [17.206887, 61.03282], [17.231615, 61.044258]]], 'type': 'Polygon'}, 'id': '039e3e653bd01f2744b1bb6f50fd2d7b'}


In [701]:
df[df['NAME_1']=='Halland']

,NAME_1,id,Deaths,Mobility,Tested_Confirmed,ICU_Patients
13,Halland,9e0b5060dac5ebe4891fc14840568d41,9,-13.0,126,10


In [703]:
import plotly.express as px



fig = px.choropleth_mapbox(df, geojson=counties, color="Mobility",
                           locations="id", #featureidkey="properties.district",
                           center = {'lat': 62.598584, 'lon': 12.961619},
                           mapbox_style="carto-positron", zoom=3.5, hover_name='NAME_1',
                           hover_data=['Deaths','Mobility','Tested_Confirmed','ICU_Patients'],
                          color_continuous_scale=plotly.colors.diverging.RdYlGn,
                          labels={'colorbar':'hi'})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()